In [1]:
import json
from unskript import nbparams
from unskript.secrets import ENV_MODE, ENV_MODE_AWS
from unskript.fwk.workflow import Task, Workflow

env = {"ENV_MODE":"ENV_MODE_AWS","TENANT_ID":"982dba5f-d9df-48ae-a5bf-ec1fc94d4882","ENVIRONMENT_ID":"1499f27c-6406-4fbd-bd1b-c6f92800018f","TENANT_URL":"https://tenant-staging.alpha.unskript.io","AWS_REGION":"us-west-2"}
secret_store_cfg = {"SECRET_STORE_TYPE":"SECRET_STORE_TYPE_AWS","AWS_SECRET_PREFIX":"staging","AWS_REGION":"us-west-2"}
paramDict = {}
paramDict.update(env)
paramDict.update(secret_store_cfg)
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)

w = Workflow(env, secret_store_cfg, None, global_vars=globals())




<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates Rollback k8s Deployment and Update Jira using unSkript legos.</b>
</div>

<br>

<center><h2>Rollback k8s Deployment and Update Jira</h2></center>

# Steps Overview
    1) Rollback the kubernaties deplayment to previous stable version.
    2) Update the Jira issue status

Here we will use unSkript Kubectl rollout deployment Lego. This lego takes k8s_cli_string, deployment and namespace as input. This inputs is used to Rollback the kubernaties deplayment to previous stable version.

In [ ]:
from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_rollout_deployment_printer(data: str):
    if data is None:
        print("Error while executing command")
        return

    print (data)

@beartype
def k8s_kubectl_rollout_deployment(handle, k8s_cli_string: str, deployment: str, namespace: str) -> str:
    k8s_cli_string = k8s_cli_string.format(deployment, namespace)
    result = handle.run_native_cmd(k8s_cli_string)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        return None
    return result.stdout


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "DevEKS",
    "credential_type": "CONNECTOR_TYPE_K8S",
    "credential_id": "f4ed0bd7-1844-471c-ba5c-453ca12ac9cb"
}''')
task.configure(inputParamsJson='''{
    "deployment": "\\"nginx-deployment\\"",
    "k8s_cli_string": "kubectl rollout undo deploy {deployment} -n {namespace}",
    "namespace": "\\"default\\""
    }''')
task.configure(outputName="Deployment_status")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_rollout_deployment, lego_printer=k8s_kubectl_rollout_deployment_printer, hdl=hdl, args=args)

Here we will use unSkript Change JIRA Issue Status Lego. This lego takes issue_id, status and transition as input. This inputs is used to Change the JIRA Issue Status.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

from jira.client import JIRA
from pydantic import BaseModel, Field
from typing import Optional
import pprint

pp = pprint.PrettyPrinter(indent=4)


from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        print('\n')
        return output
    return Printer


@legoPrinter
@beartype
def jira_issue_change_status(hdl: JIRA, issue_id: str, status: str, transition: str = ""):
    """jira_get_issue_status gets the status of a given Jira issue.
        :type issue_id: str
        :param issue_id: ID of the issue whose status we want to fetch (eg ENG-14)
        :rtype: String with issue status fetched from JIRA API
    """

    # Input param validation.
    issue = hdl.issue(issue_id)

    if transition:
        transitions = hdl.transitions(issue)
        t = [t for t in transitions if t.get('name') == status]

        if len(t) > 1:
            print("Multiple transitions possible for JIRA issue. Please select transition number to use", [
                t.get('id') for t in transitions if t.get('name') == status])
            return
        else:
            transition = t[0].get('id')

    hdl.transition_issue(issue, transition)
    return


def unskript_default_printer(output):
    if isinstance(output, (list, tuple)):
        for item in output:
            print(f'item: {item}')
    elif isinstance(output, dict):
        for item in output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(output)

task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "unSkript Jira",
    "credential_type": "CONNECTOR_TYPE_JIRA",
    "credential_id": "088d7d9b-73d1-4fe2-8579-a5d97eddfca6"
}''')
task.configure(inputParamsJson='''{
    "issue_id": "\\"PLAY-17\\"",
    "status": "\\"Done\\""
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(jira_issue_change_status, lego_printer=unskript_default_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS and Jira legos to perform AWS and Jira actions and this runbook rollback the kubernaties deplayment to the previous stable version and update the issue status in jira. To view the full platform capabilities of unSkript please visit https://unskript.com